In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
train_data = TabularDataset('../input/train.csv.gz')

In [3]:
train_data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
%time
predictor = TabularPredictor(label='label').fit(train_data, presets='good_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240921_170604"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.8
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:16:46 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T8112
CPU Count:          8
Memory Avail:       11.98 GB / 24.00 GB (49.9%)
Disk Space Avail:   1345.40 GB / 1858.19 GB (72.4%)
Presets specified: ['good_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this 

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 9.06 µs


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2024-09-21 13:06:09,961	INFO worker.py:1743 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
		Context path: "AutogluonModels/ag-20240921_170604/ds_sub_fit/sub_fit_ho"
(_dystack pid=4850) Running DyStack sub-fit ...
(_dystack pid=4850) Beginning AutoGluon training ... Time limit = 894s
(_dystack pid=4850) AutoGluon will save models to "AutogluonModels/ag-20240921_170604/ds_sub_fit/sub_fit_ho"
(_dystack pid=4850) Train Data Rows:    53333
(_dystack pid=4850) Train Data Columns: 784
(_dystack pid=4850) Label Column:       label
(_dystack pid=4850) Problem Type:       multiclass
(_dystack pid=4850) Preprocessing data ...
(_dystack pid=4850) Train Data Class Count: 10
(_dystack pid=4850) Using Feature Generators to preprocess the data ...
(_dystack pid=4850) Fitting AutoMLPi

In [5]:
test_data = TabularDataset('../input/test.csv.gz')

Loaded data from: ../input/test.csv.gz | Columns = 785 / 785 | Rows = 10000 -> 10000


In [6]:
y_pred = predictor.predict(test_data)

In [7]:
y_pred

0       7
1       2
2       1
3       0
4       4
       ..
9995    2
9996    3
9997    4
9998    5
9999    6
Name: label, Length: 10000, dtype: int64

In [8]:
test_data.label

0       7
1       2
2       1
3       0
4       4
       ..
9995    2
9996    3
9997    4
9998    5
9999    6
Name: label, Length: 10000, dtype: int64

In [9]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9735,
 'balanced_accuracy': 0.9733296531482674,
 'mcc': 0.9705466444986919}

In [10]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1_FULL,0.9777,NaN,accuracy,0.189611,NaN,36.066360,0.189611,NaN,36.066360,1,True,6
1,WeightedEnsemble_L3_FULL,0.9735,NaN,accuracy,0.295471,NaN,58.427785,0.000840,NaN,0.320332,3,True,8
2,WeightedEnsemble_L2_FULL,0.9735,NaN,accuracy,0.296606,NaN,58.579768,0.001975,NaN,0.472315,2,True,7
3,NeuralNetFastAI_BAG_L1_FULL,0.9545,NaN,accuracy,0.105020,NaN,22.041093,0.105020,NaN,22.041093,1,True,5
4,WeightedEnsemble_L3,NaN,0.98415,accuracy,NaN,6.026092,2248.018348,NaN,0.004806,0.320332,3,False,4
5,WeightedEnsemble_L2,NaN,0.98415,accuracy,NaN,6.026097,2248.170331,NaN,0.004811,0.472315,2,False,3
6,NeuralNetFastAI_BAG_L1,NaN,0.98335,accuracy,NaN,0.745283,1143.984700,NaN,0.745283,1143.984700,1,False,1
7,LightGBMXT_BAG_L1,NaN,0.96535,accuracy,NaN,5.276003,1103.713316,NaN,5.276003,1103.713316,1,False,2
